`cd project/seniorthesis/notebooks`

`conda list PyCO2SYS`

`pip install PyCO2SYS`

`pip install git+https://github.com/jbusecke/cmip6_preprocessing.git`

`mv *.png ../figures`

In [1]:
import xarray as xr
import xesmf as xe
from cmip6_preprocessing.preprocessing import combined_preprocessing
import PyCO2SYS as pyco2 # i have to install this package through terminal for it to work
import intake
import matplotlib.pyplot as plt
import numpy as np
import warnings
import gsw
from cmip6_preprocessing.postprocessing import combine_datasets
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import cmocean
import matplotlib.path as mpath
from cmip6_preprocessing.postprocessing import combine_datasets
from cmip6_preprocessing.postprocessing import merge_variables
from dask.diagnostics import ProgressBar

In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")


# `gn` datasets

1. retrieve datasets
2. preprocess
3. merge variables
4. filter for datasets that have all variables 
5. combine datasets + select first ensemble member

In [3]:
variables = ['dissic','talk', 'so', 'thetao', 'fgco2']
z_kwargs = {'consolidated': True, 'use_cftime': True}
query = dict(experiment_id=['historical'], 
             table_id=['Omon'], 
             variable_id=variables,
             grid_label=['gn'],
              source_id=['IPSL-CM6A-LR', 
#                          'MPI-ESM1-2-LR', 
                         'CNRM-ESM2-1', 
                         'CanESM5', 
#                          'MPI-ESM1-2-HR', 
#                          'CMCC-ESM2', 
#                          'GISS-E2-1-G', 
                         'UKESM1-0-LL',
#                          'ACCESS-ESM1-5', 
                         'CanESM5-CanOE' 
#                          'GISS-E2-1-G-CC',
#                          'MPI-ESM-1-2-HAM'
                        ]
            )


cat = col.search(**query)

# print(cat.df['source_id'].unique())
dd_gn = cat.to_dataset_dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'},
                                preprocess=combined_preprocessing, aggregate=False)

dd_gn = merge_variables(dd_gn)
# list(dd_new.keys())

temp={}
for name,item in dd_gn.items():
    #print(name)
    #print(item.data_vars)
    present = item.data_vars
    if all(i in present for i in variables):
        #print(name)
        temp[name]=item
dd_gn = temp


def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

dd_gn = combine_datasets(
    dd_gn,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
list(dd_gn.keys())
# dd_new_new_gn


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r2i1p1f2.Omon.gn.none.so failed to combine with :cannot compare cftime.DatetimeProlepticGregorian(1850, 1, 16, 12, 0, 0, 0) and cftime.DatetimeGregorian(1850, 1, 16, 12, 0, 0, 0) (different calendars)
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.IPSL.IPSL-CM6A-LR.historical.r10i1p1f1.Omon.gn.none.talk failed to combine with :cannot compare cftime.DatetimeGregorian(1850, 1, 16, 12, 0, 0, 0) and cftime.DatetimeProlepticGregorian(1850, 1, 16, 12, 0, 0, 0) (different calendars)
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.MOHC.UKESM1-0-LL.historical.r12i1p1f2.Omon.gn.none.fgco2 fail

['IPSL-CM6A-LR.gn.historical.Omon',
 'CNRM-ESM2-1.gn.historical.Omon',
 'CanESM5.gn.historical.Omon',
 'CanESM5-CanOE.gn.historical.Omon',
 'UKESM1-0-LL.gn.historical.Omon']

In [4]:
# Regrid
targetgrid_ds = xe.util.grid_global(1.0, 1.0)
targetgrid_ds['lon'] = targetgrid_ds['lon']+180
targetgrid_ds['lon_b'] = targetgrid_ds['lon_b']+180
newcoords = {'x':targetgrid_ds['lon'][0,:],'y':targetgrid_ds['lat'][:,0]}

dd_regrid={}
for name,item in dd_gn.items():
    regridder = xe.Regridder(item, targetgrid_ds, 'bilinear', 
                         periodic=True, ignore_degenerate=True)
    ds_regridded = regridder(item).assign_coords(**newcoords).chunk({'time':120,'lev':1})
    dd_regrid[name]=ds_regridded
list(dd_regrid.keys())

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:496: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply

['IPSL-CM6A-LR.gn.historical.Omon',
 'CNRM-ESM2-1.gn.historical.Omon',
 'CanESM5.gn.historical.Omon',
 'CanESM5-CanOE.gn.historical.Omon',
 'UKESM1-0-LL.gn.historical.Omon']

In [5]:
ds = dd_regrid['UKESM1-0-LL.gn.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('UKESM1_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('UKESM1_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('UKESM1_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('UKESM1_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('UKESM1_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [6]:
ds = dd_regrid['IPSL-CM6A-LR.gn.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('IPSL_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('IPSL_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('IPSL_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('IPSL_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('IPSL_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [7]:
ds = dd_regrid['CNRM-ESM2-1.gn.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CNRM_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CNRM_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CNRM_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CNRM_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CNRM_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [8]:
ds = dd_regrid['CanESM5-CanOE.gn.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5CanOE_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5CanOE_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5CanOE_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5CanOE_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5CanOE_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [9]:
ds = dd_regrid['CanESM5.gn.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CanESM5_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [12]:
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).nbytes/1e6

23.328

# `gr` datasets

In [10]:
variables = ['dissic','talk', 'so', 'thetao', 'fgco2']
z_kwargs = {'consolidated': True, 'use_cftime': True}
query = dict(experiment_id=['historical'], 
             table_id=['Omon'], 
             variable_id=variables,
             grid_label=['gr'],
              source_id=['GFDL-ESM4', 'CESM2-WACCM', 'CESM2', 
                         'CESM2-FV2', 'MRI-ESM2-0', 
                         'GFDL-CM4', 'CESM2-WACCM-FV2']
            )


cat = col.search(**query)

# print(cat.df['source_id'].unique())
dd_gr = cat.to_dataset_dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'},
                                preprocess=combined_preprocessing, aggregate=False)

dd_gr = merge_variables(dd_gr)

temp={}
for name,item in dd_gr.items():
    #print(name)
    #print(item.data_vars)
    present = item.data_vars
    if all(i in present for i in variables):
        #print(name)
        temp[name]=item
dd_gr = temp
# list(dd_new_gr.keys())

dd_gr = combine_datasets(
    dd_gr,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
list(dd_gr.keys())
# dd_new_new_gr


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.NCAR.CESM2-WACCM-FV2.historical.r3i1p1f1.Omon.gr.none.fgco2 failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.NCAR.CESM2.historical.r10i1p1f1.Omon.gr.none.dissic failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:138: UserWarning: CMIP.NCAR.CESM2-FV2.historical.r3i1p1f1.Omon.gr.none.talk failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/home/jovyan/.local/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:13

['CESM2-WACCM.gr.historical.Omon',
 'CESM2.gr.historical.Omon',
 'GFDL-ESM4.gr.historical.Omon',
 'MRI-ESM2-0.gr.historical.Omon',
 'CESM2-WACCM-FV2.gr.historical.Omon',
 'GFDL-CM4.gr.historical.Omon',
 'CESM2-FV2.gr.historical.Omon']

In [11]:
ds= dd_gr['CESM2-FV2.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-FV2_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-FV2_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-FV2_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-FV2_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-FV2_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [12]:
ds= dd_gr['CESM2-WACCM-FV2.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM-FV2_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM-FV2_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM-FV2_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM-FV2_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM-FV2_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [13]:
ds= dd_gr['CESM2-WACCM.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2-WACCM_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [14]:
ds= dd_gr['CESM2.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('CESM2_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [15]:
ds= dd_gr['GFDL-CM4.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-CM4_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-CM4_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-CM4_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-CM4_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-CM4_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [16]:
ds= dd_gr['GFDL-ESM4.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-ESM4_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-ESM4_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-ESM4_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-ESM4_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('GFDL-ESM4_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [ ]:
ds= dd_gr['MRI-ESM2-0.gr.historical.Omon']
ds.dissic.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('MRI-ESM2-0_dissic_20yr.nc')
ds.talk.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('MRI-ESM2-0_talk_20yr.nc')
ds.thetao.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('MRI-ESM2-0_thetao_20yr.nc')
ds.so.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('MRI-ESM2-0_so_20yr.nc')
ds.fgco2.isel(time = slice(0,239)).mean('time',keep_attrs=True).to_netcdf('MRI-ESM2-0_fgco2_20yr.nc')

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/notebook/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [9]:
dd = {**dd_gr, **dd_regrid}
list(dd.keys())

['CESM2-WACCM.gr.historical.Omon',
 'CESM2.gr.historical.Omon',
 'CESM2-WACCM-FV2.gr.historical.Omon',
 'GFDL-ESM4.gr.historical.Omon',
 'CESM2-FV2.gr.historical.Omon',
 'GFDL-CM4.gr.historical.Omon',
 'MRI-ESM2-0.gr.historical.Omon',
 'CNRM-ESM2-1.gn.historical.Omon',
 'IPSL-CM6A-LR.gn.historical.Omon',
 'CanESM5.gn.historical.Omon',
 'UKESM1-0-LL.gn.historical.Omon',
 'CanESM5-CanOE.gn.historical.Omon']